#### Example queries from Multi-omic provider.

##### This jupyter notebook give an example to query data from disease specific gene  mutation

In [1]:
import requests

import pandas as pd
def Query_fields():
    Query_str = "https://biothings.ncats.io/tcga_mut_freq_kp/metadata/fields"
    response = requests.get(Query_str)
    result = response.json()
    return(result)

def format_result_Mut_Freq(result):
    subject_id = []
    subject_symbol = []
    subject_type = []

    object_id = []
    object_name = []
    object_type = []

    edge_context = []
    edge_label = []
    edge_confidence_freq_by_case = []
    edge_confidence_freq_by_sample = []
    
    for i in range(0,len(result['hits'])):
        #Edges 
        edge_label.append(result['hits'][i]['association']['edge_label'])
        edge_confidence_freq_by_case.append(result['hits'][i]['association']['freq_by_case'])
        edge_confidence_freq_by_sample.append(result['hits'][i]['association']['freq_by_sample'])
     
        #Subject
        subject_id.append(result['hits'][i]['subject']['id'])  
        subject_symbol.append(result['hits'][i]['subject']['SYMBOL'])
        subject_type.append(result['hits'][i]['subject']['type'])

        #Object
        object_id.append(result['hits'][i]['object']['id'])
        object_name.append(result['hits'][i]['object']['tcga_name'])
        object_type.append(result['hits'][i]['object']['type'])

    #Generating dataFrame for the results
    result_df = pd.DataFrame({
    "subject_id":subject_id,
    "subject_symbol": subject_symbol,
    "subject_type": subject_type,
        
    "object_id":object_id,
    "object_name":object_name,
    "object_type":object_type,
        
    "edge_label": edge_label,
    "edge_freq_by_case": edge_confidence_freq_by_case,
    "edge_freq_by_sample":edge_confidence_freq_by_sample})
    return(result_df)


def Query_Mut_Freq_KP(Query):
    Query_str = "https://biothings.ncats.io/tcga_mut_freq_kp/query?q="
    
    count = 0
    for i in Query:
        count = count + 1
        if count != len(Query):
            Query_str = Query_str + i + ":"+ Query[i] + "%20AND%20"
        else:
            Query_str = Query_str + i + ":"+ Query[i]+"&size=1000"
    print(Query_str)
    response = requests.get(Query_str)
    result = response.json()
    return(result)

#### Browse the field info in the KG

In [2]:
#Get to know the fields in the KG
meta = Query_fields()
meta

{'association': {'index': True, 'type': 'object'},
 'association.edge_label': {'index': True, 'type': 'keyword'},
 'association.freq_by_case': {'index': True, 'type': 'float'},
 'association.freq_by_sample': {'index': True, 'type': 'float'},
 'association.no_mut_cases': {'index': True, 'type': 'integer'},
 'association.no_mut_samples': {'index': True, 'type': 'integer'},
 'association.relation_name': {'index': True, 'type': 'keyword'},
 'object': {'index': True, 'type': 'object'},
 'object.MONDO': {'index': True,
  'searched_by_default': True,
  'type': 'keyword'},
 'object.id': {'index': True, 'type': 'keyword'},
 'object.tcga_name': {'index': True, 'type': 'keyword'},
 'object.type': {'index': True, 'type': 'keyword'},
 'subject': {'index': True, 'type': 'object'},
 'subject.SYMBOL': {'index': True,
  'searched_by_default': True,
  'type': 'keyword'},
 'subject.id': {'index': True, 'type': 'keyword'},
 'subject.type': {'index': True, 'type': 'keyword'}}

####  Set up the query

In [3]:
#https://www.w3schools.com/tags/ref_urlencode.ASP

Query = {#"object.id":"%22MONDO:0018177%22",
         "association.freq_by_case":"%3E0.05",
         #"association.freq_by_case":"%3E0.1",
         "subject.SYMBOL":"EGFR"
        }

#### Query data from the drug_response_kp

In [4]:
result = Query_Mut_Freq_KP(Query)  #Jason format

https://biothings.ncats.io/tcga_mut_freq_kp/query?q=association.freq_by_case:%3E0.05%20AND%20subject.SYMBOL:EGFR&size=1000


In [5]:
result_df = format_result_Mut_Freq(result) #Pandas DataFrame format

In [6]:
result_df.sort_values(by = ['edge_freq_by_case'], ascending = False)

,subject_id,subject_symbol,subject_type,object_id,object_name,object_type,edge_label,edge_freq_by_case,edge_freq_by_sample
0,SYMBOL:EGFR,EGFR,gene,MONDO:0018177,GBM,disease,gene_has_variant_that_contributes_to_disease_a...,0.234097,0.235732
2,SYMBOL:EGFR,EGFR,gene,MONDO:0005061,LUAD,disease,gene_has_variant_that_contributes_to_disease_a...,0.126984,0.126538
4,SYMBOL:EGFR,EGFR,gene,MONDO:0005012,SKCM,disease,gene_has_variant_that_contributes_to_disease_a...,0.072961,0.072650
3,SYMBOL:EGFR,EGFR,gene,MONDO:0005499,LGG,disease,gene_has_variant_that_contributes_to_disease_a...,0.070312,0.068441
1,SYMBOL:EGFR,EGFR,gene,MONDO:0000553,UCEC,disease,gene_has_variant_that_contributes_to_disease_a...,0.067925,0.067797
